In [1]:
import pandas as pd

# File path of the Excel file
file_path = r'C:\Users\MICHAELBOYLE\OneDrive - Deloitte (O365D)\Desktop\Datasets for python\April 2023 world economic outlook.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(file_path)



In [2]:
# Get the current column headers
headers = df.columns.tolist()

# Generate the new column headers for the specified years
new_years = [str(year) for year in range(1980, 2029)]
new_headers = headers[:9] + new_years + headers[-1:]

# Assign the new column headers back to the dataframe
df.columns = new_headers

# Print the modified dataframe headers
print(df.columns)


Index(['WEO Country Code', 'ISO', 'WEO Subject Code', 'Country',
       'Subject Descriptor', 'Subject Notes', 'Units', 'Scale',
       'Country/Series-specific Notes', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028',
       'Estimates Start After'],
      dtype='object')


In [3]:
# Filter the dataframe based on 'Country Name' and 'Subject Descriptor'
filtered_df = df[(df['Country'] == 'United Kingdom') & (df['Subject Descriptor'] == 'Gross domestic product, deflator')]

# Access the time series data for the filtered columns
time_series_data = filtered_df.iloc[:, 9:]

In [4]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objs as go

import pandas as pd
import numpy as np

# assuming you have a pandas dataframe named df
# df = pd.read_csv('your_data.csv')  # replace 'your_data.csv' with your actual csv file

# Create the Dash app
app = JupyterDash(__name__)

available_countries = ['Germany', 'United Kingdom', 'Brazil']
available_years = [str(year) for year in range(1980, 2029)]

app.layout = html.Div([
    html.Label('Country'),
    dcc.Dropdown(
        id='country_dropdown',
        options=[{'label': i, 'value': i} for i in available_countries],
        value=available_countries[0]
    ),
    html.Label('Start Year'),
    dcc.Dropdown(
        id='starting_year_dropdown',
        options=[{'label': i, 'value': i} for i in available_years],
        value=available_years[0]
    ),
    dcc.Graph(id='my-graph')
])

@app.callback(
    Output('my-graph', 'figure'),
    [Input('country_dropdown', 'value'), 
     Input('starting_year_dropdown', 'value')])
def update_graph(country, starting_year):
    starting_year = int(starting_year)
    subject_descriptor = "Gross domestic product, constant prices"

    # Filter the dataframe based on 'Country' and 'Subject Descriptor'
    filtered_df = df[(df['Country'] == country) & (df['Subject Descriptor'] == subject_descriptor)]

    # Access the time series data for the filtered columns
    time_series_data = filtered_df.iloc[:, 9:-1]

    # Get the column headers as the years
    years = time_series_data.columns.astype(int).tolist()

    # Calculate the base value for the chosen starting year
    base_index = years.index(starting_year)
    base_value = time_series_data.iloc[:, base_index].values

    # Divide the time series data by the base value and multiply by 100
    normalized_data = (time_series_data.values / base_value[:, np.newaxis]) * 100

    # Find the index of the first estimate year
    estimate_index = years.index(2023)

    # Generate your chart
    fig = go.Figure()

    # Add actual data line (including 2023)
    fig.add_trace(go.Scatter(
        x = years[base_index:estimate_index+1], 
        y = normalized_data[0][base_index:estimate_index+1], 
        mode='lines',
        line=dict(color='blue', width=2),
        name='Actual'
    ))

    # Add estimated data line (starting from 2023)
    fig.add_trace(go.Scatter(
        x = years[estimate_index:], 
        y = normalized_data[0][estimate_index:], 
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),  # Change to a dashed line
        name='Estimated'
    ))

    fig.update_layout(
        title='GDP Deflator Time Series for {} (Base Year: {})'.format(country, starting_year),
        xaxis_title='Year',
        yaxis_title='Normalised GDP Deflator',
        annotations=[
            dict(
                x=0.01,
                y=-0.2,
                showarrow=False,
                text="Source: International Monetary Fund (IMF), World Economic Outlook April 2023",
                xref="paper",
                yref="paper"
            )
        ]
    )
    
    return fig





app.run_server(mode='external')


Dash is running on http://127.0.0.1:8050/



C:\Users\MICHAE~1\AppData\Local\Temp/ipykernel_23072/3276734095.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\MICHAE~1\AppData\Local\Temp/ipykernel_23072/3276734095.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Dash app running on http://127.0.0.1:8050/
